In [1]:
import sqlite3
import numpy as np
import pandas as pd
from typing import Dict, Optional, Tuple
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor

import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool
from itertools import product


In [2]:
# Connect to database
conn = sqlite3.connect('rossmann.db')
cursor = conn.cursor()

# Print table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('rossmann',)]


In [3]:
# Assign 'rossmann' table to Pandas DataFrame
sql = "SELECT * FROM rossmann"
df = pd.read_sql(sql, conn)

In [4]:
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(["Store", "Date"])

In [5]:
df['CompetitionDistance'] = df['CompetitionDistance'].fillna(df['CompetitionDistance'].median())
df = df.drop(columns=['CompetitionOpenSinceMonth', 'Promo2SinceWeek', 'SchoolHoliday'])

In [6]:
df[['CompetitionOpenSinceYear', 'Promo2SinceYear']] = df[['CompetitionOpenSinceYear', 'Promo2SinceYear']].fillna(0, axis=1)
df['PromoInterval'] = df['PromoInterval'].fillna('Unknown')

In [7]:
from sklearn.preprocessing import MinMaxScaler

cols_to_scale = [
    "CompetitionDistance",
    # "CompetitionOpenSinceMonth",
    "CompetitionOpenSinceYear",
    # "Promo2SinceWeek",
    "Promo2SinceYear",
    "Customers"
]

scaler = MinMaxScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

In [8]:
# ============================================================
# 2. FIXED ONE-HOT ENCODING (NOT DYNAMIC)
# ============================================================
onehot_cols = [
    "StoreType", 
    "PromoInterval",
    "Assortment",
    "DayOfWeek"
]

prefix_map = {
    "StoreType": "StoreType_",
    "PromoInterval": "PromoInterval_",
    "Assortment": "Assortment_",
    "DayOfWeek": "DayOfWeek_"
}

for col in onehot_cols:
    if col in df.columns:
        df = pd.concat([
            df,
            pd.get_dummies(df[col], prefix=prefix_map[col], dtype=int)
        ], axis=1)

df = df.drop(columns=[c for c in onehot_cols if c in df.columns])


# ============================================================
# 3. GLOBAL CONFIG
# ============================================================
DEFAULT_STEPS = 54
DEFAULT_MIN_LEN = 60
LAGS = (1, 7, 14, 28)

onehot_prefixes = ("StoreType_", "PromoInterval_", "Assortment_", "DayOfWeek_")



In [9]:
df.head()

,index,Store,Date,Sales,Customers,Open,Promo,StateHoliday,CompetitionDistance,CompetitionOpenSinceYear,...,Assortment__a,Assortment__b,Assortment__c,DayOfWeek__1,DayOfWeek__2,DayOfWeek__3,DayOfWeek__4,DayOfWeek__5,DayOfWeek__6,DayOfWeek__7
1016095,1016095,1,2013-01-01,0,0.000000,0,0,a,0.016482,0.996526,...,1,0,0,0,1,0,0,0,0,0
1014980,1014980,1,2013-01-02,5530,0.090417,1,0,0,0.016482,0.996526,...,1,0,0,0,0,1,0,0,0,0
1013865,1013865,1,2013-01-03,4327,0.078235,1,0,0,0.016482,0.996526,...,1,0,0,0,0,0,1,0,0,0
1012750,1012750,1,2013-01-04,4486,0.083785,1,0,0,0.016482,0.996526,...,1,0,0,0,0,0,0,1,0,0
1011635,1011635,1,2013-01-05,4997,0.085950,1,0,0,0.016482,0.996526,...,1,0,0,0,0,0,0,0,1,0


In [10]:
# ============================================================
# 4. METRIC FUNCTIONS
# ============================================================
def _safe_rmse(y, yhat):
    return float(np.sqrt(mean_squared_error(y, yhat)))

def _mape(y, yhat):
    y = np.asarray(y, dtype=float)
    yhat = np.asarray(yhat, dtype=float)
    mask = y != 0
    return float(np.mean(np.abs((yhat[mask] - y[mask]) / y[mask])))

def _tae(y, yhat):
    y = np.asarray(y, dtype=float)
    yhat = np.asarray(yhat, dtype=float)
    mask = y != 0
    return float(np.abs((yhat[mask].sum() - y[mask].sum())))



In [11]:

# ============================================================
# 5. FEATURE BUILDER (EXACT SAME AS SARIMAX NOTEBOOK)
# ============================================================
def _make_features(
    work: pd.DataFrame,
    target_col: str = "Sales",
    lags=LAGS,
):
    df = work.copy()
    df = df.set_index("Date").sort_index()
    # print(df.head())

    lag_cols = []
    for lag in lags:
        col = f"{target_col}_lag{lag}"
        df[col] = df[target_col].shift(lag)
        lag_cols.append(col)
    # print(df.head())

    df["dow"] = df.index.dayofweek
    df["month"] = df.index.month
    df["year"] = df.index.year
    df["weekofyear"] = df.index.isocalendar().week.astype(int)
    date_cols = ["dow", "month", "year", "weekofyear"]

    onehot = [c for c in df.columns if c.startswith(onehot_prefixes)]
    binary = [c for c in ["Open", "Promo", "SchoolHoliday", "Promo2"] if c in df.columns]
    cont = [
        c for c in [
            "Sales","CompetitionDistance",
            "CompetitionOpenSinceMonth","CompetitionOpenSinceYear",
            "Promo2SinceWeek","Promo2SinceYear"
        ] if c in df.columns
    ]

    feature_cols = onehot + binary + cont + lag_cols + date_cols
    feature_cols = [c for c in feature_cols if c != target_col]

    X = df[feature_cols]
    y = df[target_col]
    # print(X.head())
    

    mask = X.notna().all(axis=1) & y.notna()
    print(y[mask].head())
    return X[mask], y[mask]

In [12]:

# ============================================================
# 6. GPU GRID SEARCH (single store)
# ============================================================
def grid_search_xgb_gpu(
    work: pd.DataFrame,
    param_grid: Dict[str, list],
    steps: int = DEFAULT_STEPS,
    min_len: int = DEFAULT_MIN_LEN,
    metric: str = "rmse",
):
    X, y = _make_features(work)
    print(y.head())

    if len(y) <= steps + min_len:
        print("not enough rows")
        print(len(y))
        print(steps)
        return None

    split_idx = len(y) - steps
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    keys = list(param_grid.keys())
    combos = list(product(*[param_grid[k] for k in keys]))

    best_score = np.inf
    best_params = None
    best_model = None
    best_forecast = None
    best_metrics = None

    for combo in combos:
        params = dict(zip(keys, combo))

        params["objective"] = "reg:squarederror"
        # params["tree_method"] = "gpu_hist"
        # params["predictor"] = "gpu_predictor"

        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        yhat = model.predict(X_test)

        metrics = {
            "mae": float(mean_absolute_error(y_test, yhat)),
            "rmse": _safe_rmse(y_test, yhat),
            "mape": _mape(y_test, yhat),
            "tae": _tae(y_test, yhat),
        }

        score = metrics[metric]

        if score < best_score:
            best_score = score
            best_params = params
            best_model = model
            best_metrics = metrics
            best_forecast = pd.DataFrame(
                {"y_true": y_test.values, "y_pred": yhat},
                index=y_test.index,
            )

    return {
        "best_params": best_params,
        "best_metrics": best_metrics,
        "best_model": best_model,
        "best_forecast": best_forecast,
    }

In [13]:
# ============================================================
# 7. FIND GLOBAL BEST PARAMS ON ONE SELECTED STORE
# ============================================================
TARGET_STORE = 1

param_grid = {
    "max_depth": [4, 6, 8],
    "learning_rate": [0.05, 0.1],
    "n_estimators": [300, 500],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
}

df_target = df[df["Store"] == TARGET_STORE]
df_target.head()

,index,Store,Date,Sales,Customers,Open,Promo,StateHoliday,CompetitionDistance,CompetitionOpenSinceYear,...,Assortment__a,Assortment__b,Assortment__c,DayOfWeek__1,DayOfWeek__2,DayOfWeek__3,DayOfWeek__4,DayOfWeek__5,DayOfWeek__6,DayOfWeek__7
1016095,1016095,1,2013-01-01,0,0.000000,0,0,a,0.016482,0.996526,...,1,0,0,0,1,0,0,0,0,0
1014980,1014980,1,2013-01-02,5530,0.090417,1,0,0,0.016482,0.996526,...,1,0,0,0,0,1,0,0,0,0
1013865,1013865,1,2013-01-03,4327,0.078235,1,0,0,0.016482,0.996526,...,1,0,0,0,0,0,1,0,0,0
1012750,1012750,1,2013-01-04,4486,0.083785,1,0,0,0.016482,0.996526,...,1,0,0,0,0,0,0,1,0,0
1011635,1011635,1,2013-01-05,4997,0.085950,1,0,0,0.016482,0.996526,...,1,0,0,0,0,0,0,0,1,0


In [14]:
best = grid_search_xgb_gpu(df_target, param_grid)
best_params = best["best_params"]

print("Best Params Found on Store", TARGET_STORE, ":\n", best_params)

Date
2013-01-29    3725
2013-01-30    4601
2013-01-31    4709
2013-02-01    5633
2013-02-02    5970
Name: Sales, dtype: int64
Date
2013-01-29    3725
2013-01-30    4601
2013-01-31    4709
2013-02-01    5633
2013-02-02    5970
Name: Sales, dtype: int64
Best Params Found on Store 1 :
 {'max_depth': 4, 'learning_rate': 0.05, 'n_estimators': 300, 'subsample': 1.0, 'colsample_bytree': 0.8, 'objective': 'reg:squarederror'}


In [15]:
# ============================================================
# 8. TRAIN ALL STORES USING THE GLOBAL BEST PARAMS (GPU)
# ============================================================
def fit_store_xgb(work, steps, min_len, xgb_kwargs):
    X, y = _make_features(work)
    if len(y) <= steps + min_len:
        return None

    split_idx = len(y) - steps
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train,  y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    model = XGBRegressor(**xgb_kwargs)
    model.fit(X_train, y_train)

    yhat = model.predict(X_test)

    metrics = {
        "mae": float(mean_absolute_error(y_test, yhat)),
        "rmse": _safe_rmse(y_test, yhat),
        "mape": _mape(y_test, yhat),
        "tae": _tae(y_test, yhat),
    }

    forecast = pd.DataFrame(
        {"y_true": y_test.values, "y_pred": yhat},
        index=y_test.index,
    )

    return {"model": model, "metrics": metrics, "forecast": forecast}


def run_store_xgb(
    store_id, df_all, best_params, steps=DEFAULT_STEPS, min_len=DEFAULT_MIN_LEN
):
    df_store = df_all[df_all["Store"] == store_id].copy()

    params = best_params.copy()
    params["objective"] = "reg:squarederror"
    # params["tree_method"] = "gpu_hist"
    # params["predictor"] = "gpu_predictor"

    out = fit_store_xgb(df_store, steps, min_len, params)

    if out is None:
        return (store_id, None, None)
    return (store_id, out["metrics"], out["forecast"])

In [16]:
stores = df["Store"].unique()
n_threads = multiprocessing.cpu_count()

args = [(s, df, best_params, DEFAULT_STEPS, DEFAULT_MIN_LEN) for s in stores]

with ThreadPool(n_threads) as pool:
    results = pool.starmap(run_store_xgb, args)

Date
2013-01-29    3725
2013-01-30    4601
2013-01-31    4709
2013-02-01    5633
2013-02-02    5970
Name: Sales, dtype: int64
Date
2013-01-29    12178
2013-01-30    11494
2013-01-31    13414
2013-02-01    14160
2013-02-02    10489
Name: Sales, dtype: int64
Date
2013-01-29    7195
2013-01-30    7943
2013-01-31    8376
2013-02-01    8734
2013-02-02    9677
Name: Sales, dtype: int64
Date
2013-01-29    6562
2013-01-30    6586
2013-01-31    7404
2013-02-01    8772
2013-02-02    7167
Name: Sales, dtype: int64
Date
2013-01-29    3863
2013-01-30    4176
2013-01-31    5767
2013-02-01    5126
2013-02-02    3055
Name: Sales, dtype: int64
Date
2013-01-29    5110
2013-01-30    5217
2013-01-31    7850
2013-02-01    7450
2013-02-02    6505
Name: Sales, dtype: int64
Date
2013-01-29    8721
2013-01-30    8299
2013-01-31    9023
2013-02-01    9986
2013-02-02    5706
Name: Sales, dtype: int64
Date
2013-01-29    3708
2013-01-30    4044
2013-01-31    5146
2013-02-01    4840
2013-02-02    4533
Name: Sales, 

In [17]:
# ============================================================
# 9. AGGREGATE METRICS + FORECASTS
# ============================================================
metrics_rows = []
forecast_list = []

for store_id, metrics, fc in results:
    if metrics is None:
        continue

    m = metrics.copy()
    m["Store"] = store_id
    metrics_rows.append(m)

    f = fc.copy()
    f["Store"] = store_id
    forecast_list.append(f)

df_metrics = pd.DataFrame(metrics_rows).set_index("Store").sort_index()

df_forecasts = (
    pd.concat(forecast_list)
      .reset_index()
      .rename(columns={"index": "Date"})
      .sort_values(["Store", "Date"])
)

df_metrics.to_csv("xgb_store_metrics.csv")
df_forecasts.to_csv("xgb_store_forecasts.csv", index=False)

print("Done")

Done


In [19]:
# Load forecast-level data
df_forecast = pd.read_csv("xgb_store_forecasts.csv")   # Date, y_true, y_pred, Store

# Drop rows with missing actuals if any
df_forecast = df_forecast.dropna(subset=["y_true", "y_pred"])

# ---------- 1) Aggregated WAPE ----------
# WAPE_org = sum_i |y_hat_i - y_i| / sum_i |y_i|
df_forecast["abs_err"] = (df_forecast["y_pred"] - df_forecast["y_true"]).abs()
wape_org = df_forecast["abs_err"].sum() / df_forecast["y_true"].abs().sum()

# ---------- 2) Aggregated weighted MAPE ----------
# Weighted MAPE_org with weights proportional to actuals:
# w_i = y_i / sum_j y_j  →  wMAPE = sum_i w_i * |err_i|/|y_i|
# This simplifies to the same formula as WAPE but on relative errors:
mask = df_forecast["y_true"] != 0
g = df_forecast[mask].copy()

weights = g["y_true"].abs() / g["y_true"].abs().sum()
rel_err = (g["y_pred"] - g["y_true"]).abs() / g["y_true"].abs()
weighted_mape_org = (weights * rel_err).sum()

print("Aggregated WAPE:", wape_org)
print("Aggregated weighted MAPE:", weighted_mape_org)

Aggregated WAPE: 0.08736437246335264
Aggregated weighted MAPE: 0.0860964998323813
